In [2]:
import requests
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from random import randint

https://blog.hartleybrody.com/web-scraping-cheat-sheet/

# Get Wikipedia article content

In [8]:
import wikipedia as wiki

In [10]:
museum_df_f = '/Users/klarnemann/Documents/Insight/Insight_project/data/scraped_museums_museumusa.xlsx'
museum_df = pd.read_excel(museum_df_f, index_col=0)
museum_df['Article_content'] = ''
museum_df.shape

(11542, 6)

In [11]:
museum_df.head()

,Name,State,City,Category,Description,Article_content
0,Aldrich Coal Mining Museum,Alabama,Aldrich,NaN,NaN,
1,Horseshoe Bend National Military Park,Alabama,Daviston,Park,NaN,
2,Bayou La Batre-Coden Historical Foundation,Alabama,Irvington,NaN,NaN,
3,Autauga County Heritage Association,Alabama,Prattville,"Archaeology, Cemetery, Church, Culture, Histor...",The Autauga County Heritage Association is a g...,
4,Buena Vista Mansion,Alabama,Prattville,"Culture, History, Historic House",Premier Event Venue in the River Region! A bea...,


In [12]:
for i, title in enumerate(museum_df.Name):
    try:
        museum_df.loc[i, 'Article_content'] = wiki.page(title).content
    except:
        continue
    if i % 100 == 0:
        print(i)
        museum_df.to_excel(museum_df_f)

0


/Users/klarnemann/anaconda3/lib/python3.7/site-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/klarnemann/anaconda3/lib/python3.7/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


100
200
300
400
600
700
900
1000
1100
1200
1300
1400
1500
1800
1900
2000
2100
2200
2300
2400
2600
2800
2900
3100
3200
3300
3500
3700
3800
3900
4000
4100
4200
4300
4400
4500
4700
4800
4900
5000
5200
5300
5400
5500
5600
5700
5800
5900
6100
6200
6400
6500
6600
6700
6800
6900
7100
7200
7300
7400
7500
7600
7700
7800
7900
8100
8200
8300
8400
8500
8600
8700
8800
8900
9100
9200
9300
9400
9500
9600
9700
9800
9900
10100
10200
10400
10500
10700
10800
10900
11000
11100
11200
11300
11400
11500


In [13]:
museum_df.head()

,Name,State,City,Category,Description,Article_content
0,Aldrich Coal Mining Museum,Alabama,Aldrich,NaN,NaN,The Aldrich Coal Mine Museum is a historical m...
1,Horseshoe Bend National Military Park,Alabama,Daviston,Park,NaN,Horseshoe Bend National Military Park is a U.S...
2,Bayou La Batre-Coden Historical Foundation,Alabama,Irvington,NaN,NaN,
3,Autauga County Heritage Association,Alabama,Prattville,"Archaeology, Cemetery, Church, Culture, Histor...",The Autauga County Heritage Association is a g...,"The Montgomery–Janes–Whittaker House, best kno..."
4,Buena Vista Mansion,Alabama,Prattville,"Culture, History, Historic House",Premier Event Venue in the River Region! A bea...,


# Scrape MuseumUSA

In [3]:
main_url = 'http://www.museumsusa.org'

In [4]:
def scrape_directories(url, category):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    outputs = []
    output_urls = []
    for item in soup.find_all(class_='browseCategoryItem'):
        try:
            item_url = item.find_all('a')[0].get('href')
            if category in item_url:
                output_url = '%s/%s' % (main_url, item_url)
                output_urls.append(output_url)
                val, _ = item.find_all('a')[0].contents[0].split(' (')
                outputs.append(val)
        except:
            pass
    return outputs, output_urls

In [5]:
def scrape_museum_content(url):
    t0 = time.time()
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    response_time = time.time() - t0
    # get site name
    names = []
    for name in soup.find_all(class_='source'):
        names.append(name.find_all('a')[0].contents[0])
    # get site category
    categories = []
    for category in soup.find_all(class_='type'):
        try:
            categories.append(category.contents[0])
        except:
            categories.append('')
    # get site description
    descriptions = []
    for description in soup.find_all(class_='abstract'):
        try:
            descriptions.append(description.contents[0])
        except:
            descriptions.append('')
    return names, categories, descriptions, response_time

In [6]:
def scrape_museums(city, state, city_url, museum_df):
    names, categories, descriptions, response_time = scrape_museum_content(city_url)
    time.sleep(10*response_time)#delay next request
    n_items = len(names)
    tmp_dat = np.array([names, [state]*n_items, [city]*n_items, categories, descriptions], dtype=object).T
    tmp_df = pd.DataFrame(tmp_dat, columns=museum_df.columns)
    return tmp_df

In [7]:
def run_scrape_content(state, state_url, museum_df, stop=None):
    print('Scrapping %s museums...' % (state))
    cities, city_urls = scrape_directories(state_url, 'City')
    n_cities = len(cities)
    if type(stop) == type(None):
        stop = n_cities
    for i, city_url in enumerate(city_urls):
        if i > stop:
            return museum_df
        if i % 20 == 0:
            print(i)
        city = cities[i]
        if city in museum_df.City.unique():
            continue
        tmp_df = scrape_museums(city, state, city_url, museum_df)
        museum_df = museum_df.append(tmp_df, ignore_index=True)
    return museum_df

# Run

In [9]:
museum_df = pd.read_excel('/Users/klarnemann/Documents/Insight/Insight_project/data/scraped_museums_museumusa.xlsx', index_col=0)
museum_df.shape

(10360, 5)

In [10]:
museum_df.head()

,Name,State,City,Category,Description
0,Aldrich Coal Mining Museum,Alabama,Aldrich,NaN,NaN
1,Horseshoe Bend National Military Park,Alabama,Daviston,Park,NaN
2,Bayou La Batre-Coden Historical Foundation,Alabama,Irvington,NaN,NaN
3,Autauga County Heritage Association,Alabama,Prattville,"Archaeology, Cemetery, Church, Culture, Histor...",The Autauga County Heritage Association is a g...
4,Buena Vista Mansion,Alabama,Prattville,"Culture, History, Historic House",Premier Event Venue in the River Region! A bea...


In [20]:
np.sum(~pd.isna(museum_df.Description))

5493

In [21]:
np.sum(~pd.isna(museum_df.Category))

10072

~~~
states, state_urls = scrape_directories('http://www.museumsusa.org/museums/?k=1271392%2cDirectoryID%3a200454', 'State')
~~~

~~~
for state in states:
    ind = states.index(state)
    state_url = state_urls[ind]
    museum_df = run_scrape_content(state, state_url, museum_df)
    museum_df.to_excel('/Users/klarnemann/Documents/Insight/Insight_project/data/scraped_museums_museumusa.xlsx')
~~~

~~~
columns = ['Name', 'State', 'City', 'Category', 'Description']
for i, state_url in enumerate(state_urls):
    if i < 16:
        continue
    state = np.array(states)[i]
    print(state)
    t0 = time.time()#start time
    cities, city_urls = scrape_directories(state_url, 'City')
    response_delay = time.time() - t0#request duration
    time.sleep(10*response_delay)#delay next request
    for j, city_url in enumerate(np.array(city_urls)):
        if j % 20 == 0:
            print(j)
        city = cities[j]
        if state == 'California':
            if city in ca_cities:
                continue
        t0 = time.time()#start time
        names, categories, descriptions = scrape_content(city_url)
        response_delay = time.time() - t0#request duration
        time.sleep(10*response_delay)#delay next request
        n_items = len(names)
        tmp_dat = np.array([names, [state]*n_items, [city]*n_items, categories, descriptions], dtype=object).T
        tmp_df = pd.DataFrame(tmp_dat, columns=columns)
        museum_df = museum_df.append(tmp_df, ignore_index=True)
~~~

## Scrape States

In [5]:
test_url = 'http://www.museumsusa.org/museums/?k=1271392%2cDirectoryID%3a200454'
test_page = requests.get(test_url)
soup = BeautifulSoup(test_page.text, 'html.parser')

In [6]:
soup.find_all(class_='browseCategoryItem')[1].find_all('a')[0]

<a href="/museums/?k=1271400%2cState%3aAL%3bDirectoryID%3a200454">Alabama (324)</a>

In [7]:
soup.find_all(class_='browseCategoryItem')[1].find_all('a')[0].get('href')

'/museums/?k=1271400%2cState%3aAL%3bDirectoryID%3a200454'

In [8]:
# select second site name
state = soup.find_all(class_='browseCategoryItem')[1].find_all('a')[0].contents[0]
print(state)

Alabama (324)


In [9]:
states, state_urls = scrape_directories('http://www.museumsusa.org/museums/?k=1271392%2cDirectoryID%3a200454', 'State')

## Scrape Cities (given state)

In [10]:
test_url = state_urls[0]
test_page = requests.get(test_url)
soup = BeautifulSoup(test_page.text, 'html.parser')

In [11]:
soup.find_all(class_='browseCategoryItem')[1].find_all('a')[0]

<a href="/museums/?k=1271404%2cCity%3aAldrich%3bState%3aAL%3bDirectoryID%3a200454">Aldrich (1)</a>

In [12]:
# select second site name
state = soup.find_all(class_='browseCategoryItem')[1].find_all('a')[0].contents[0]
print(state)

Aldrich (1)


In [13]:
soup.find_all(class_='browseCategoryItem')[1].find_all('a')[0].get('href')

'/museums/?k=1271404%2cCity%3aAldrich%3bState%3aAL%3bDirectoryID%3a200454'

In [36]:
cities, city_urls = scrape_directories(state_urls[0], 'City')
cities[:5]

['Aldrich', 'Daviston', 'Irvington', 'Prattville', 'Aliceville']

## Scrape content (given city, state)

### Example: Amsterdam, NY

In [49]:
# setup parser
test_url = city_urls[3]
test_page = requests.get(test_url)
soup = BeautifulSoup(test_page.text, 'html.parser')

In [50]:
soup.find(class_='source').find_all('a')

[<a href="/museums/info/3293">Autauga County Heritage Association</a>,
 <a href="http://www.autaugaheritage.org" title="Organization Web Page"><img alt="Organization Web Page" src="/i/external_link.gif"/></a>]

In [51]:
# get first site category
soup.find(class_='type').contents[0]

'Archaeology, Cemetery, Church, Culture, History, Historic House, Historical Society, Library, Military'

In [52]:
# get first site location
soup.find(class_='location').contents[0]

'Prattville, Alabama'

In [31]:
# scrape example location
#scrape_content(test_url)